In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
import sklearn.metrics

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K


from pathlib import Path
import os.path

Using TensorFlow backend.


In [23]:
PID = 42
smartphone = "N5X"
extension = 0

sensors = {
    "acc",
    "gyro",
    "ori",
    "grav",
    "mag",
    "rot"
}

tasks = {
    "points",
    "fitts"
}

file_names = sensors.union(tasks)

raw_data = dict()

for file in file_names:
    file_path = str(Path.home()) + "/data/fapra_imu-" +  str(PID) + "-" + file + "-" + smartphone + "-" + str(extension) + ".csv"
    if not os.path.isfile(file_path):
        print(file_path + "not found")
        continue
    raw_data[file] = pd.read_csv(file_path, ";")

In [24]:
time_filtered_data = dict()
for name in file_names:
    tmp = []
    # remove [1:] if both lenght are equal (first press is removed)
    for k, end in enumerate(raw_data["points"].time[1:]):
        start = int(raw_data["fitts"].time[k])
        mask = (raw_data[name]["time"] > start) & (raw_data[name]["time"] <= end)
        tmp.append(raw_data[name].loc[mask])
    time_filtered_data[name] = tmp

In [26]:
for k, item in enumerate(time_filtered_data["points"][0]):
    interval = dict()
    for sensor in file_names:
        interval[sensor] = time_filtered_data[sensor][k]
    # first timestamp included
    begin_interval = raw_data["fitts"].time[k] + 1
    # find smallest timestamp
    # create array of all values of given timestamp
    print(begin_interval)
    print(interval)

1543421184498
{'rot':               time         x         y         z
500  1543421184519  0.349784  0.214654  0.532768
501  1543421184519  0.350159  0.214606  0.532573
502  1543421184519  0.350554  0.214651  0.532307
503  1543421184520  0.350936  0.214827  0.531962
504  1543421184520  0.351317  0.215091  0.531588
505  1543421184520  0.351709  0.215364  0.531180
506  1543421184520  0.352120  0.215536  0.530718
507  1543421184520  0.352520  0.215615  0.530234
508  1543421184538  0.352821  0.215643  0.529792
509  1543421184538  0.353022  0.215649  0.529411
510  1543421184538  0.353134  0.215583  0.529110
511  1543421184538  0.353163  0.215484  0.528875
512  1543421184557  0.353094  0.215367  0.528712
513  1543421184558  0.352961  0.215247  0.528600
514  1543421184558  0.352779  0.215157  0.528523
515  1543421184558  0.352552  0.215154  0.528468
516  1543421184578  0.352278  0.215266  0.528432
517  1543421184578  0.351997  0.215482  0.528399
518  1543421184578  0.351728  0.215784  0.52834

{'rot':                time         x         y         z
2004  1543421192022  0.256938  0.320222  0.650454
2005  1543421192022  0.257420  0.320534  0.650481
2006  1543421192022  0.258128  0.320756  0.650459
2007  1543421192022  0.258983  0.320863  0.650397
2008  1543421192045  0.259942  0.320842  0.650289
2009  1543421192045  0.260935  0.320683  0.650156
2010  1543421192045  0.261922  0.320381  0.649978
2011  1543421192045  0.262840  0.319905  0.649753
2012  1543421192064  0.263699  0.319265  0.649474
2013  1543421192064  0.264453  0.318480  0.649158
2014  1543421192064  0.265054  0.317556  0.648817
2015  1543421192064  0.265507  0.316569  0.648435
2016  1543421192082  0.265810  0.315583  0.648030
2017  1543421192082  0.265986  0.314661  0.647598
2018  1543421192082  0.266032  0.313806  0.647149
2019  1543421192082  0.265998  0.313059  0.646710
2020  1543421192102  0.265918  0.312381  0.646259
2021  1543421192102  0.265852  0.311774  0.645811
2022  1543421192102  0.265821  0.311228  0

In [8]:
max_features = 1024

model = Sequential()
model.add(Embedding(max_features, output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)

ValueError: Error when checking input: expected embedding_2_input to have 2 dimensions, but got array with shape (60000, 28, 28)